In [11]:
from tamr_unify_client import Client
from tamr_unify_client.auth import UsernamePasswordAuth
import pandas as pd
import json

In [12]:
# need to be set by the user
cluster_id = [
    '3f479c09-631a-36da-80d0-17fcd42f0b94',
    '53726262-ed75-363f-bc29-fe11609ceb86',
    '1b20685b-4cb7-313b-b117-1e634d114d98',
    '41c064e7-eae4-305b-ac2f-b378be3b5555',
    '52191e5f-d70b-3144-9e31-f41bf11dd617',
    '465e5a91-ef00-3fe1-85e3-dbc253791a16',
    '478fa661-5269-35cc-94ef-7e9459ffae83',
    'bf43f1d7-c7f1-3751-84cd-da0353577186',
    '67b6f3a0-c144-38ed-80ac-cbcee5089b17',
    'f80ef4d2-091f-34e1-8be7-27a7d9216d78'
 
             ]
proj_name = 'Provider Mastering'
ud_name ='Provider Mastering_unified_dataset'



In [13]:
auth = UsernamePasswordAuth('admin','production')
unify_client = Client(auth, host='10.20.1.230', base_path='/api/')
def query_hbase(unify_client, record_info, ds_name):
    url = f"dataset/datasets/{ds_name}/records"
    resp = unify_client.post(url, json=record_info)
    if resp.status_code >= 300:
        print(resp.text)
    resp.raise_for_status()
    return resp.json()

In [14]:
test=[]
for m in cluster_id:
    cluster_toquery = {'persistentId': m}
    ds_name = f'{ud_name}_dedup_published_cluster_stats'
    cluster_info = query_hbase(unify_client, cluster_toquery, ds_name)
    cluster_info
   
    cluster_members = cluster_info['recordIds']
    cluster_members
    records_toquery = [{"tamr_id": val["entityId"]} for val in cluster_members]
    record_info = []
    for rec in records_toquery:
        record_info.append(query_hbase(unify_client, rec, ud_name))
        all_recs = {rec['tamr_id']: rec for rec in record_info}
        keysList = list(all_recs.keys())
        all_recs[keysList[0]]
        test.append(all_recs[keysList[0]])
print(len(test))

710


In [59]:
print(test[0])   
import collections.abc
import numpy as np
df = pd.DataFrame(test)
#print(df)
#df = df.applymap(lambda x: x[0] if (len(x) > 0)) # If same operation is to be used on all columns
df = df.applymap(lambda x: x[0]  if ( isinstance(x, collections.abc.Sequence)and len(x) >= 1) else x)
df

{'origin_source_name': 'Physician_Compare_National_batch_1.csv', 'origin_entity_id': '42019', 'tamr_id': '-1005341558464751826', 'Credentials': ['MD'], 'HCPCS_CODE': None, 'HCPCS_DESCRIPTION': None, 'GroupNPI': None, 'NPPES_Entity_Type': None, 'NPPES_NPI': ['1013972538'], 'TaxID': None, 'VendorName': None, 'Vendor_Street_Address': None, 'Provider_First_Name': ['AZHAR'], 'Provider_Last_Name': ['SHEIKH', 'SOUTHEASTERN PHYSICIAN SERVICES PC'], 'Provider_Full_Name': ['AZHAR SHEIKH'], 'VendorState': None, 'VendorZip': None, 'Provider_Street_Address_Line1': ['8249 STANDIFER GAP RD'], 'Provider_Street_Address_Line2': None, 'Provider_Type': None, 'Provider_Mailing_State': ['TN'], 'Provider_Mailing_Postal_Code': None, 'Provider_Mailing_City': ['CHATTANOOGA'], 'Provider_Gender': ['M'], 'Provider_Taxonomy_Code': None, 'Provider_Speciality_Type': ['INTERNAL MEDICINE'], 'NPPES_Provider_Organization_Name': [''], 'Enrollment Status Effective Date': None, 'Payment': 0.0, 'Last Update Date': None, 'Ven

,origin_source_name,origin_entity_id,tamr_id,Credentials,HCPCS_CODE,HCPCS_DESCRIPTION,GroupNPI,NPPES_Entity_Type,NPPES_NPI,TaxID,...,NPPES_Practice_Location_Address,NPPES_Practice_Location_City,NPPES_Practice_Location_Postal_Code,NPPES_Practice_Location_State,NPPES_Reference_Mailing_State,NPPES_Provider_Enumeration_Date,NPPES_Last_Updated_Date,NPPES_Deactivated_Date,NPPES_Provider_Gender_Code,NPPES_Provider_Taxonomy_Code
0,P,4,-,MD,None,None,None,None,1013972538,None,...,1060 WILLIAM WAY NW,CLEVELAND,373124369,TN,TN,04/18/2006,05/02/2016,,M,207R00000X
1,P,4,-,MD,None,None,None,None,1013972538,None,...,1060 WILLIAM WAY NW,CLEVELAND,373124369,TN,TN,04/18/2006,05/02/2016,,M,207R00000X
2,P,4,-,MD,None,None,None,None,1013972538,None,...,1060 WILLIAM WAY NW,CLEVELAND,373124369,TN,TN,04/18/2006,05/02/2016,,M,207R00000X
3,P,4,-,MD,None,None,None,None,1013972538,None,...,1060 WILLIAM WAY NW,CLEVELAND,373124369,TN,TN,04/18/2006,05/02/2016,,M,207R00000X
4,P,4,-,MD,None,None,None,None,1013972538,None,...,1060 WILLIAM WAY NW,CLEVELAND,373124369,TN,TN,04/18/2006,05/02/2016,,M,207R00000X
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
705,O,-,-,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
706,O,-,-,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
707,O,-,-,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
708,O,-,-,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [60]:
df.to_csv("providerMasteringCompare.csv")


In [62]:
import random;

mf = pd.read_csv("providerMastering.csv",on_bad_lines='skip')
for i in range(0,70):
    df.loc[i, 'Provider_Street_Address_Line1'] = '796, Main,St'
    df.loc[i, 'Provider_Mailing_State'] = 'MA'
    df.loc[i, 'Provider_Mailing_City'] = 'Acton'
    df.loc[i,'Provider_Mailing_Postal_Code'] = '1720'
    df.loc[i, 'Payment'] =  random.randrange(250, 600)

    
for i in range(70,100):
    df.loc[i, 'Provider_Street_Address_Line1'] = '796 St'
    df.loc[i, 'Provider_Mailing_State'] = 'MA'
    df.loc[i, 'Provider_Mailing_City'] = 'Acton'
    df.loc[i,'Provider_Mailing_Postal_Code'] = '1720'
    
for i in range(100,125):
    df.loc[i, 'Provider_Street_Address_Line1'] = '796 ave'
    df.loc[i, 'Provider_Mailing_State'] = 'MA'
    df.loc[i, 'Provider_Mailing_City'] = 'Acton'
    df.loc[i,'Provider_Mailing_Postal_Code'] = '1720'

for i in range(125,150):
    df.loc[i, 'Provider_Street_Address_Line1'] = '796, Main,Straet'
    df.loc[i, 'Provider_Mailing_State'] = 'MA'
    df.loc[i, 'Provider_Mailing_City'] = 'Acton'
    df.loc[i,'Provider_Mailing_Postal_Code'] = '1720'

for i in range(150,191):
    df.loc[i, 'Provider_Street_Address_Line1'] = '796, Main,Street'
    df.loc[i, 'Provider_Mailing_State'] = 'MA'
    df.loc[i, 'Provider_Mailing_City'] = 'Acton'
    df.loc[i,'Provider_Mailing_Postal_Code'] = '1720'
    
for i in range(0,50):
    df.loc[i,'HCPCS_DESCRIPTION'] = 'Enteral and Parenteral Therapy'
    df.loc[i,'HCPCS_CODE'] ='B4034'
    df.loc[i,'GroupNPI'] ='0123456A'
    df.loc[i,'Enrollment Status Effective Date'] = '01/12/09'
    
    


In [63]:
for i in range(192,252):
    df.loc[i, 'Provider_Street_Address_Line1'] = '767,5th,Ave'
    df.loc[i, 'Provider_Mailing_State'] = 'NY'
    df.loc[i, 'Provider_Mailing_City'] = 'New York'
    df.loc[i,'Provider_Mailing_Postal_Code'] = '10153'
    df.loc[i, 'Payment'] =  random.randrange(250, 600)

    
for i in range(252,272):
    df.loc[i, 'Provider_Street_Address_Line1'] = '5th,Ave'
    df.loc[i, 'Provider_Mailing_State'] = 'NY'
    df.loc[i, 'Provider_Mailing_City'] = 'New York'
    df.loc[i,'Provider_Mailing_Postal_Code'] = '10153'
    
for i in range(252,292):
    df.loc[i, 'Provider_Street_Address_Line1'] = '767,5th,Avenue'
    df.loc[i, 'Provider_Mailing_State'] = 'NY'
    df.loc[i, 'Provider_Mailing_City'] = 'New York'
    df.loc[i,'Provider_Mailing_Postal_Code'] = '10153'
    
for i in range(292,303):
    df.loc[i, 'Provider_Street_Address_Line1'] = '767,5th,Ave'
    df.loc[i, 'Provider_Mailing_State'] = 'NY'
    df.loc[i, 'Provider_Mailing_City'] = 'New York'
    df.loc[i,'Provider_Mailing_Postal_Code'] = '10153'

    
for i in range(192,252):
    df.loc[i,'HCPCS_DESCRIPTION'] = 'Behavioral health screening to determine eligibility for admission to treatment program'
    df.loc[i,'HCPCS_CODE'] ='F0002'
    df.loc[i,'GroupNPI'] ='014531B'
    df.loc[i,'Enrollment Status Effective Date'] = '05/08/08'
    
df.to_csv("providerMastering.csv", index=False)

    

In [ ]:
for i in range(304,354):
    df.loc[i, 'Provider_Street_Address_Line1'] = '65, Warbler Way'
    df.loc[i, 'Provider_Mailing_State'] = 'IN'
    df.loc[i, 'Provider_Mailing_City'] = 'Indianapolis'
    df.loc[i,'Provider_Mailing_Postal_Code'] = '46256'
    df.loc[i, 'Payment'] =  random.randrange(250, 600)

for i in range(354,374):
    df.loc[i, 'Provider_Street_Address_Line1'] = 'Warbler Way'
    df.loc[i, 'Provider_Mailing_State'] = 'IN'
    df.loc[i, 'Provider_Mailing_City'] = 'Indianapolis'
    df.loc[i,'Provider_Mailing_Postal_Code'] = '46256'
    
for i in range(374,384):
    df.loc[i, 'Provider_Street_Address_Line1'] = '65 Warbler'
    df.loc[i, 'Provider_Mailing_State'] = 'IN'
    df.loc[i, 'Provider_Mailing_City'] = 'Indianapolis'
    df.loc[i,'Provider_Mailing_Postal_Code'] = '46256'
    
for i in range(304,354):
    df.loc[i,'HCPCS_DESCRIPTION'] = 'Behavioral health screening to determine eligibility for admission to treatment program'
    df.loc[i,'HCPCS_CODE'] ='F0002'
    df.loc[i,'GroupNPI'] ='014531B'
    df.loc[i,'Enrollment Status Effective Date'] = '03/18/03'